# CGI - Step 3: Inverting the targeted gradient

In [ ]:
%load_ext autoreload
%autoreload 2

## Lib

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
import torchvision.utils as vutils
from torchvision import models, datasets, transforms
from collections import defaultdict, OrderedDict
from copy import deepcopy
import re
import copy
import time
import math
import logging

from torch.utils.data import Dataset, DataLoader
from torchvision import models, utils, datasets, transforms
from torchvision.datasets.utils import verify_str_arg
from torchvision.datasets.utils import download_and_extract_archive
import numpy as np
import sys
import os
from PIL import Image

In [ ]:
import global_var

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_printoptions(8,sci_mode=True)

global_var.set_device(device)

In [ ]:
torch.cuda.is_available()

In [ ]:
from dataset import *
from common_DL import *
from gradient_lib import *
from federated_learning import *
from model_structure import *
from utils import *
from inversion_attacks import *
from model_structure import *

from CGI_framework_lib3 import *

In [ ]:
!pip install lpips

## Dataset

In [ ]:
Cifar100_train_dataset = get_dataset("Cifar100",train=True,transform=None,download=True)
Cifar100_test_dataset= get_dataset("Cifar100",train=False,transform=None,download=True)

Cifar100_train_loader = torch.utils.data.DataLoader(Cifar100_train_dataset,batch_size=128,shuffle=True)
Cifar100_test_loader = torch.utils.data.DataLoader(Cifar100_test_dataset,batch_size=128,shuffle=False)

In [ ]:
TinyImageNet_train_dataset = get_dataset("TinyImageNet",train=True,transform=None,download=True)
TinyImageNet_test_dataset= get_dataset("TinyImageNet",train=False,transform=None,download=True)

TinyImageNet_train_loader = torch.utils.data.DataLoader(TinyImageNet_train_dataset,batch_size=32,shuffle=True)
TinyImageNet_test_loader = torch.utils.data.DataLoader(TinyImageNet_test_dataset,batch_size=32,shuffle=False)

In [ ]:
CalTech256_train_dataset = get_dataset("CalTech256",train=True,transform=None,download=True)
CalTech256_test_dataset= get_dataset("CalTech256",train=False,transform=None,download=True)

CalTech256_train_loader = torch.utils.data.DataLoader(CalTech256_train_dataset,batch_size=16,shuffle=True)
CalTech256_test_loader = torch.utils.data.DataLoader(CalTech256_test_dataset,batch_size=16,shuffle=False)

In [ ]:
test_dataset_dict={
    "TinyImageNet":TinyImageNet_test_dataset,
    "Cifar100":Cifar100_test_dataset,
    "CalTech256":CalTech256_test_dataset
}

train_dataset_dict={
    "TinyImageNet":TinyImageNet_train_dataset,
    "Cifar100":Cifar100_train_dataset,
    "CalTech256":CalTech256_train_dataset
}

In [ ]:
test_loader_dict={
    "TinyImageNet":TinyImageNet_test_loader,
    "Cifar100":Cifar100_test_loader,
    "CalTech256":CalTech256_test_loader
}

train_loader_dict={
    "TinyImageNet":TinyImageNet_train_loader,
    "Cifar100":Cifar100_train_loader,
    "CalTech256":CalTech256_train_loader
}

In [ ]:
data_info = {
    "test_dataset_dict":test_dataset_dict,
    "train_dataset_dict":train_dataset_dict,
    "test_loader_dict":test_loader_dict,
    "train_loader_dict":train_loader_dict
}

## Exp

In [ ]:
settings = [({
        "batch_size":32,
        "client_num":50,
        "malicious_num":10
    },0,"Cifar100","FK","MKrum",
        {"optimization_round":1000,
        "threshold":[8e+3,1e2,3e+4],
        "init_value":[1,0],
        "lr":1e-7,
        "opt_parameter":"gamma"},
    False)]
fl_setting, target_class,dataset_name,knowledge_level,agr_name,agr_attack_setting_dict,sp = settings[0]

In [ ]:
gradient_tuple = gradient_simulate(dataset_name,data_info,target_class,knowledge_level,agr_name,fl_setting,agr_attack_setting_dict,sp)

In [ ]:
inversion(dataset_name,target_class,knowledge_level,agr_name,gradient_tuple,inversion_args=[1e-1,1e-2,10])